In [1]:
import altair as alt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def makeDotPlot(df,saveName='DotPlot.png',title='Macrophage Exercise Score, Post v Pre',
                x='ROC-AUC',y='Sample Name',color='Exercise Type',size='-Log10 P value',opac='Significant',rev=True,save=True):
    chart=alt.Chart(df).mark_circle().encode(x=alt.X(x,scale=alt.Scale(domain=[0, 1])),
                                             y=y,
                                             color=color,
                                             opacity=opac,
                                             size=alt.Size(size,
                                                           scale=alt.Scale(range=[100,500],
                                                                           reverse=rev))).properties(title=title)
    if save:
        chart.save(saveName)
    return chart

In [3]:
df=pd.read_csv('Patterns_Data.txt',sep='\t')
df['-Log10 P value']=-np.log10(df['Pval'])
df['Significant']=df['Pval']<=0.05
df.index=df['Sample Name']
data2=pd.read_csv('ExerciseInfo.csv',index_col=0)

## Figure 4 A

In [5]:
resLst=list(data2[data2['Exercise Type']=='resistance'].index)
saveName='MacrophageExerciseDotPlot_Resistance_Training_log10Pval.svg'
title='Macrophage Exercise Score, Resistance Exercise Post v Pre'
save=True
dfIn=df.loc[resLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

In [6]:
endurLst=list(data2[data2['Exercise Type']=='endurance'].index)
saveName='MacrophageExerciseDotPlot_Endurance_Training_log10Pval.svg'
title='Macrophage Exercise Score, Endurance Exercise Post v Pre'
save=True
dfIn=df.loc[endurLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

## Figure 4 B 

In [7]:
bloodLst=list(data2[data2['Method']=='Blood Draw'].index)
saveName='MacrophageExerciseDotPlot_BloodSample_log10Pval.svg'
title='Macrophage Exercise Score, PBMC Blood Post v Pre'
save=True
dfIn=df.loc[bloodLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

In [8]:
df2Lst=['GSE104079','GSE165630','GSE1718','GSE71972','GSE9103','GSE97084','GSE99963']
saveName='MacrophageExerciseDotPlot_MuscleSample_log10Pval.svg'
title='Macrophage Exercise Score, Muscle Biopsy Post v Pre'
save=True
dfIn=df.loc[df2Lst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

## Figure 4 C

In [9]:
humanDf=['GSE1718','GSE1832','GSE28498','GSE34788','GSE44051']
saveName='MacrophageExerciseDotPlot_Human_log10Pval.svg'
title='Macrophage Exercise Score, Human Post v Pre'
save=True
dfIn=df.loc[humanDf]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

In [10]:
mouseLst=list(data2[data2['Species']=='Mus musculus'].index)
mouseLst.extend(list(data2[data2['Species']=='Rattus norvegicus'].index))

saveName='MacrophageExerciseDotPlot_Mouse_log10Pval.svg'
title='Macrophage Exercise Score, Mouse Post v Pre'
save=True
dfIn=df.loc[mouseLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

## Figure 4 D

In [11]:
rnaSeqLst=list(data2[data2['Experiment Type']=='high throughput sequencing'].index)
saveName='MacrophageExerciseDotPlot_RNASeq_log10Pval.svg'
title='Macrophage Exercise Score, RNA-Seq Post v Pre'
save=True
dfIn=df.loc[rnaSeqLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

In [12]:
arrayDf=['GSE104079','GSE117525','GSE122671','GSE16907','GSE28498','GSE44051','GSE9103']
saveName='MacrophageExerciseDotPlot_MicroArray_log10Pval.svg'
title='Macrophage Exercise Score, Microarray Post v Pre'
save=True
dfIn=df.loc[arrayDf]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)